In [1]:
import pandas as pd
from sklearn.metrics import average_precision_score

# TDC package
from tdc.single_pred import Tox

# ML model pacakage
from DeepPurpose import utils as dp_utils, CompoundPred

# hyperparams package
from ray import tune
from ray.tune.search.ax import AxSearch
from ray import air
from ray.air.callbacks.mlflow import MLflowLoggerCallback

def prepare_data(drug_encoding):
    return dp_utils.data_process(
        X_drug=X, y=y, drug_encoding=drug_encoding, random_seed=RANDOM_SEED
    )


def train_fn(hparams, optimal = False):
        
    if drug_encoding == "Morgan":
        config = dp_utils.generate_config(
            drug_encoding=hparams["drug_encoding"],
            train_epoch=int(hparams["train_epochs"]),
            LR=hparams["lr"],
            batch_size=int(hparams["batch_size"]),
            mlp_hidden_dims_drug=[int(hparams["mlp_hidden_dims_drug"])] * int(hparams["mlp_num_layers_drug"])
        )
       
    elif drug_encoding == 'CNN':
        filter_base_size = int(hparams["cnn_drug_filters_base"])
        kernel_base_size = int(hparams["cnn_drug_kernels_base"])

        config = dp_utils.generate_config(
            drug_encoding=hparams["drug_encoding"],
            train_epoch=int(hparams["train_epochs"]),
            LR=hparams["lr"],
            batch_size=int(hparams["batch_size"]),
            cnn_drug_filters=[filter_base_size, filter_base_size*2, filter_base_size*3],
            cnn_drug_kernels=[kernel_base_size, kernel_base_size*2, kernel_base_size*3]
        )
        
    elif drug_encoding == 'DGL_GCN':
        config = dp_utils.generate_config(
            drug_encoding=hparams["drug_encoding"],
            train_epoch=int(hparams["train_epochs"]),
            LR=hparams["lr"],
            batch_size=int(hparams["batch_size"]),
            gnn_hid_dim_drug=int(hparams["gnn_hid_dim_drug"]),
            gnn_num_layers=int(hparams["gnn_num_layers"])
        )
        
    elif drug_encoding == 'DGL_NeuralFP':
        
        config = dp_utils.generate_config(
            drug_encoding=hparams["drug_encoding"],
            train_epoch=int(hparams["train_epochs"]),
            LR=hparams["lr"],
            batch_size=int(hparams["batch_size"]),
            gnn_hid_dim_drug=int(hparams["gnn_hid_dim_drug"]),
            gnn_num_layers=int(hparams["gnn_num_layers"]),
            neuralfp_max_degree=int(hparams["neuralfp_max_degree"]),
            neuralfp_predictor_hid_dim=int(hparams["neuralfp_predictor_hid_dim"])
        )
        
    elif drug_encoding == 'DGL_AttentiveFP':
        config = dp_utils.generate_config(
            drug_encoding=hparams["drug_encoding"],
            train_epoch=int(hparams["train_epochs"]),
            LR=hparams["lr"],
            batch_size=int(hparams["batch_size"]),
            gnn_hid_dim_drug=int(hparams["gnn_hid_dim_drug"]),
            gnn_num_layers=int(hparams["gnn_num_layers"]),
            attentivefp_num_timesteps=int(hparams["attentivefp_num_timesteps"])
        )
        
    config['device'] = 'cuda:0'
    model = CompoundPred.model_initialize(**config)
    model.train(train, val, test, verbose = False)
    
    if optimal:
        model.save_model('_'.join([drug_encoding, data_name, 'optimal']))
        scores = model.predict(test, verbose = False)
        return {'pr_auc': average_precision_score(test.Label.values, scores)}
    else:
        scores = model.predict(val, verbose = False)
        return {'pr_auc': average_precision_score(val.Label.values, scores)}

data_name = "herg_karim"
drug_encoding = "Morgan"

X, y = Tox(name=data_name).get_data(format="DeepPurpose")
train, val, test = prepare_data(drug_encoding)

search_space = {
    "drug_encoding": drug_encoding,
    "lr": tune.loguniform(1e-4, 1e-2),
    "batch_size": tune.choice([32, 64, 128, 256, 512]),
    "train_epochs": tune.choice([3,5,10,15,20])
}

if drug_encoding == "Morgan":
    search_space.update({
        "mlp_hidden_dims_drug": tune.choice([32, 64, 128, 256, 512]),
        "mlp_num_layers_drug": tune.choice([1,2,3,4,5,6])
    })
elif drug_encoding == 'CNN':
    search_space.update({
        "cnn_drug_filters_base": tune.choice([8, 16, 32, 64, 128]),
        "cnn_drug_kernels_base": tune.choice([2, 4, 6, 8, 12])
    })
elif drug_encoding == 'DGL_GCN':
    search_space.update({
        "gnn_hid_dim_drug": tune.choice([8, 16, 32, 64, 128, 256, 512]),
        "gnn_num_layers": tune.choice([1,2,3,4])
    })
elif drug_encoding == 'DGL_NeuralFP':
    search_space.update({
       "gnn_hid_dim_drug": tune.choice([8, 16, 32, 64, 128, 256, 512]),
       "gnn_num_layers": tune.choice([1,2,3,4]),
       "neuralfp_max_degree": tune.choice([5, 10, 20]),
       "neuralfp_predictor_hid_dim": tune.choice([8, 16, 32, 64, 128, 256, 512])
    })    
elif drug_encoding == 'DGL_AttentiveFP':
    search_space.update({
       "gnn_hid_dim_drug": tune.choice([8, 16, 32, 64, 128, 256, 512]),
        "gnn_num_layers": tune.choice([1,2,3,4]),
        "attentivefp_num_timesteps": tune.choice([2,3,4,5])
    })

ax_search = AxSearch(metric="pr_auc", mode="max")

tuner = tune.Tuner(
    tune.with_resources(
            tune.with_parameters(train_fn),
            resources={"gpu": 1}
        ),
    tune_config=tune.TuneConfig(search_alg=ax_search, num_samples=100),
    run_config=air.RunConfig(
        name="mlflow",
        callbacks=[
            MLflowLoggerCallback(
                tracking_uri="./mlruns",
                experiment_name="test",
                save_artifact=True,
            )
        ],
    ),
    param_space=search_space,
)

analysis = tuner.fit()

/tmp/user/21201/ipykernel_2646108/2353896950.py:22: DeprecationWarning: The module `ray.air.callbacks.mlflow` has been moved to `ray.air.integrations.mlflow` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.air.callbacks.mlflow` with `ray.air.integrations.mlflow`.
  from ray.air.callbacks.mlflow import MLflowLoggerCallback
Found local copy...
Loading...
Done!
2022-12-27 21:38:39,917	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
[INFO 12-27 21:38:57] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 12-27 21:38:57] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter batch_size. If that is not the expected value type, you can explicity specify 'value_

[INFO 12-27 21:38:58] ax.service.ax_client: Generated new trial 0 with parameters {'lr': 0.003591, 'batch_size': 512, 'train_epochs': 3, 'mlp_hidden_dims_drug': 256, 'mlp_num_layers_drug': 6, 'drug_encoding': 'Morgan'}.
(pid=2659134) WARNING:root:No normalization for BCUT2D_MWHI
(pid=2659134) WARNING:root:No normalization for BCUT2D_MWLOW
(pid=2659134) WARNING:root:No normalization for BCUT2D_CHGHI
(pid=2659134) WARNING:root:No normalization for BCUT2D_CHGLO
(pid=2659134) WARNING:root:No normalization for BCUT2D_LOGPHI
(pid=2659134) WARNING:root:No normalization for BCUT2D_LOGPLOW
(pid=2659134) WARNING:root:No normalization for BCUT2D_MRHI
(pid=2659134) WARNING:root:No normalization for BCUT2D_MRLOW
[INFO 12-27 21:40:13] ax.service.ax_client: Generated new trial 1 with parameters {'lr': 0.001331, 'batch_size': 64, 'train_epochs': 3, 'mlp_hidden_dims_drug': 256, 'mlp_num_layers_drug': 6, 'drug_encoding': 'Morgan'}.


(train_fn pid=2659134) Drug Property Prediction Mode...
(train_fn pid=2659134) in total: 13445 drugs
(train_fn pid=2659134) encoding drug...
(train_fn pid=2659134) unique drugs: 13445


(pid=2660059) WARNING:root:No normalization for BCUT2D_MWHI
(pid=2660059) WARNING:root:No normalization for BCUT2D_MWLOW
(pid=2660059) WARNING:root:No normalization for BCUT2D_CHGHI
(pid=2660059) WARNING:root:No normalization for BCUT2D_CHGLO
(pid=2660059) WARNING:root:No normalization for BCUT2D_LOGPHI
(pid=2660059) WARNING:root:No normalization for BCUT2D_LOGPLOW
(pid=2660059) WARNING:root:No normalization for BCUT2D_MRHI
(pid=2660059) WARNING:root:No normalization for BCUT2D_MRLOW


(train_fn pid=2659134) Done.


...

(train_fn pid=1010139) Drug Property Prediction Mode...
(train_fn pid=1010139) in total: 13445 drugs
(train_fn pid=1010139) encoding drug...
(train_fn pid=1010139) unique drugs: 13445


2022-12-29 04:13:35,211	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:13:40,406	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:13:45,616	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:13:50,800	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f

(train_fn pid=1010139) Done.


2022-12-29 04:13:55,951	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:14:01,181	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:14:06,400	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:14:11,669	WARNING trial_runner.py:988 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f

2022-12-29 04:16:15,768	WARNING tune.py:734 -- Trial Runner checkpointing failed: Can't pickle <function TriangularLinearOperator.to_dense at 0x7f44d279b9d0>: it's not the same object as linear_operator.operators.triangular_linear_operator.TriangularLinearOperator.to_dense
2022-12-29 04:16:15,945	INFO tune.py:762 -- Total run time: 110238.00 seconds (110237.55 seconds for the tuning loop).


In [2]:
analysis.get_dataframe()

,pr_auc,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,...,timesteps_since_restore,iterations_since_restore,warmup_time,config/batch_size,config/drug_encoding,config/lr,config/mlp_hidden_dims_drug,config/mlp_num_layers_drug,config/train_epochs,logdir
0,0.774180,152.357938,False,NaN,NaN,1,0af17128,0e954833ca2c4aa9a3fa633fab55d368,2022-12-27_21-42-45,1672206165,...,0,1,0.006095,512,Morgan,0.003591,256,6,3,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
1,0.885447,159.482431,False,NaN,NaN,1,c5611cc7,39b06fc321e749efbbd71c36e83b88c9,2022-12-27_21-44-08,1672206248,...,0,1,0.018776,64,Morgan,0.001331,256,6,3,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
2,0.903953,311.915093,False,NaN,NaN,1,ddbe9a21,97d4433dcc024f9ab6fe491ba859f618,2022-12-27_21-47-52,1672206472,...,0,1,0.004144,256,Morgan,0.000785,256,3,20,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
3,0.878732,180.985882,False,NaN,NaN,1,2e9e560a,3a5ecf034f9f4cb0884d318024ea170d,2022-12-27_21-46-47,1672206407,...,0,1,0.009746,64,Morgan,0.009384,32,5,5,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
4,0.897036,218.991350,False,NaN,NaN,1,5f268803,1a7376687be2482e8e110e2bacae6bba,2022-12-27_21-48-29,1672206509,...,0,1,0.021581,32,Morgan,0.000108,128,4,5,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.912117,179.353838,False,NaN,NaN,1,e96bba87,0210c503c86a425ebdaf874070ee9d79,2022-12-29_03-31-37,1672313497,...,0,1,0.005414,256,Morgan,0.000283,512,3,20,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
96,0.904071,193.386474,False,NaN,NaN,1,0bc1059d,0436ddc8fe9445fcafde266f42faef20,2022-12-29_03-48-59,1672314539,...,0,1,0.003896,256,Morgan,0.000273,512,3,20,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
97,0.903492,193.948227,False,NaN,NaN,1,8071165d,34893211aae04a68847f7ea3247ac489,2022-12-29_04-00-32,1672315232,...,0,1,0.010853,256,Morgan,0.000271,512,3,20,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...
98,0.913610,164.813221,False,NaN,NaN,1,c9c7d065,09ca9bc4e0104c91b9fe9bc08b2d42f4,2022-12-29_04-08-52,1672315732,...,0,1,0.004201,256,Morgan,0.000284,512,3,20,/afs/cs.stanford.edu/u/kexinh/ray_results/mlfl...


In [11]:
df = analysis.get_dataframe()
df.to_csv('_'.join([drug_encoding, data_name, 'tune_df']) + '.csv')

In [13]:
optimal_hparam = dict(df.sort_values('pr_auc').iloc[-1])
optimal_hparam = {i[7:] if i[:7] == 'config/' else i: j for i,j in optimal_hparam.items()}

In [16]:
train_fn(optimal_hparam, optimal = True)

Drug Property Prediction Mode...
in total: 13445 drugs
encoding drug...
unique drugs: 13445
Done.


{'pr_auc': 0.9153994697022456}

In [42]:
from huggingface_hub import create_repo
from huggingface_hub import HfApi, snapshot_download, hf_hub_download
import os

deeppurpose_repo = [
    'hERG_Karim_Morgan'
]


class tdc_hf_interface:
    def __init__(self, repo_name):
        self.repo_id = "tdc/" + repo_name
        
    def upload(self, folder_path):
        create_repo(repo_id=self.repo_id)
        api = HfApi()
        api.upload_folder(
            folder_path=folder_path,
            path_in_repo="model",
            repo_id=self.repo_id,
            repo_type="model"
        )
        
    def file_download(self, save_path, filename):
        model_ckpt = hf_hub_download(
            repo_id = self.repo_id,
            filename = filename,
            cache_dir = save_path
        )
        
    def repo_download(self, save_path):
        snapshot_download(repo_id=self.repo_id, cache_dir= save_path)
        
    def load_deeppurpose(self, save_path):
        if self.repo_id[4:] in deeppurpose_repo:
            save_path = save_path + '/' + self.repo_id[4:]
            if not os.path.exists(save_path):
                os.mkdir(save_path)
            self.file_download(save_path, "model/model.pt")
            self.file_download(save_path, "model/config.pkl")
            
            save_path = save_path + '/models--tdc--' + self.repo_id[4:] + '/blobs/'
            file_name1 = save_path + os.listdir(save_path)[0]
            file_name2 = save_path + os.listdir(save_path)[1]
            
            if os.path.getsize(file_name1) > os.path.getsize(file_name2):
                model_file, config_file = file_name1, file_name2
            else:
                config_file, model_file = file_name1, file_name2

            os.rename(model_file, save_path + 'model.pt')
            os.rename(config_file, save_path + 'config.pkl')            
            
            from DeepPurpose import CompoundPred
            net = CompoundPred.model_pretrained(path_dir = save_path)
            return net
        else:
            raise ValueError("This repo does not host a DeepPurpose model!")

In [43]:
tdc_hf_herg = tdc_hf_interface("hERG_Karim_Morgan")

In [3]:
tdc_hf_herg.upload('./Morgan_herg_karim_optimal')

In [44]:
net = tdc_hf_herg.load_deeppurpose('./data')

Downloading:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]